## Extract tables

In [2]:
import re
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

In [3]:
from docx import Document
import json


def extract_data(file_path):
    doc = Document(file_path)
    data = []
    date = doc.paragraphs[1].text.strip()
    date = re.sub(r'[а-яА-Я\s]', '', date)
    for table in doc.tables:
        for row in table.rows[1:]:  # Пропускаем заголовок
            cells = row.cells
            if cells[2].text.strip() == "" or cells[3].text.strip() == "":
                continue
            if "костюм" in preprocess_text(cells[0].text.strip()):
                type_clothes = "костюм"
            else:
                type_clothes = "other"
            
            item = {
                "id": len(data) + 1,  # Генерация ID
                "name": cells[0].text.strip(),
                "type":type_clothes,
                "date":date,
                "description": preprocess_text(f"{cells[0].text.strip()} {cells[1].text.strip()} {cells[2].text.strip()}"),
                "price": cells[3].text.strip(),  # Преобразуем в число
                "currency": "руб.",
                "source": file_path
            }
            data.append(item)
    
    return data

In [5]:
# Сохранение в JSON
import os 
from pathlib import Path


docs = os.listdir('КП')
for doc in docs:
    filepath = f"КП/{doc}"
    data = extract_data(filepath)
    if data!=[]:
        with open(f"json_db/{Path(filepath).stem}.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)


## vector db

In [6]:
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from uuid import uuid4
from langchain.document_loaders import JSONLoader



def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name")
    metadata["price"] = record.get("price")
    metadata["currency"] = record.get("currency")
    metadata["type"] = record.get("type")
    metadata["date"] = record.get("date")

    return metadata


uuids = [str(uuid4()) for _ in range(len(os.listdir("json_db")))]
docs = []
for file in os.listdir("json_db"):

    file_name = "json_db/" + file

    loader = JSONLoader(
        file_path=file_name,
        jq_schema=".[]",
        content_key="description",
        metadata_func=metadata_func
        #text_content=False
    )
    document = loader.load()
    docs += document


In [7]:
from langchain_chroma import Chroma

model_name = "DeepPavlov/rubert-base-cased"
embedding_model = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-base")

vector_store = Chroma(
    collection_name="json_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

/tmp/ipykernel_20771/3572408945.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-base")
/home/wormsin/miniconda3/envs/genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
uuids = [str(uuid4()) for _ in range(len(docs))]

In [ ]:
vector_store.add_documents(documents=docs, ids = uuids)

## Retriever

In [16]:
retriever = vector_store.as_retriever(search_type = "mmr", search_kwargs={"filter": {"type": "костюм"}})

In [11]:
def format_docs(docs):
    return "\n\n".join([f"Date: {doc.metadata['date']}\nPrice: {doc.metadata['price'] + doc.metadata['currency']}\nContent: {doc.page_content}" for doc in docs])

## RAG chain

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")

In [37]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


template = """
Задача:
Оценить цену нового продукта на основе предоставленного контекста.

Инструкции:
Пользователь предоставит описание нового продукта.
Вам нужно будет использовать цены существующих продуктов из поля "Price" и их описания из поля "Content".
Поле "Date" указывает, когда продукт был предложен по указанной цене.
Если указана дата, скорректируйте цену с учётом инфляции, увеличив цену на 10% за каждый год в зависимости от разницы между текущей датой и датой контекста.
Если поле "Date" отсутствует или пустое, не вносите изменений в цену.
Чётко объясните, какие параметры повлияли на оценочную цену(перечисли определенные характеристика изделия, если таковые есть и их влияние) 
и как (например, увеличили или уменьшили стоимость).
Предоставьте конкретную числовую цену, если это возможно, с учётом инфляции, если это применимо.
Если точную цену определить невозможно, чётко объясните, почему, какие дополнительные данные нужны и какие параметры отсутствуют.
Не выдумывайте информацию. Если в контексте нет релевантных данных, просто ответьте: "Я не знаю." с указанием причины. 


<context>
{context}
</context>

question: {question}

"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

llm_chain = prompt | llm | StrOutputParser()

In [34]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

rag_chain = {"context":retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()} | llm_chain

In [35]:
description = """КУРТКА:
- прямого силуэта
- с центральной супатной застёжкой на пять пуговиц и две сквозные вверху борта
- полочки и спинка с кокетками из отделочной ткани
- по кокеткам проложена СОП шириной 50 мм
- полочки с накладными карманами: одним верхними на левой полочке и двумя нижними
- верхний накладной карман с клапаном, застёгивающимся на ленты контакт
- нижние накладные карманы с наклонным входом
- рукава втачные, с трикотажной манжетой"""


In [ ]:
print(rag_chain.invoke(description))

Для оценки цены нового продукта — куртки, описанной вами, я рассмотрел предоставленный контекст и выбрал наиболее подходящие существующие продукты.

1. **Первый продукт:** Женский костюм "бренд 1"
   - **Цена:** 3010 руб.
   - **Дата:** 03.04.2023
   - **Описание:** Женский костюм с курткой, в описании упоминаются элементы, такие как застёжка на молнию, карманы и использование смесовых тканей, но кроме этого, куртка имеет значительное количество деталей, которые отличаются от вашего описания.

2. **Второй продукт:** Костюм сварщика 3 кл.
   - **Цена:** 10620 руб.
   - **Дата:** 26.01.2023
   - **Описание:** Костюм с защитными элементами, выполненный из 100% хлопка, что делает его более специализированным изделием. Цена здесь значительно выше, чем у других костюмов.

3. **Третий продукт:** Костюм «стандарт»
   - **Цена:** 1200 руб.
   - **Дата:** 31.03.2023
   - **Описание:** Простая куртка и брюки, но тоже не соответствует требованиям, указанным в вашем новом описании.

4. **Четвёртый 